In [ ]:
from utils.calvin_styles import *
from influxdb import DataFrameClient
from utils.libs import *
from utils.user import *
from itertools import combinations

In [ ]:
import plotly.graph_objs as go

host = 'influxdb.gsg.capital'
port = 8086
db_fxtrading_md = DataFrameClient(host, port, influx_id['user'], influx_id['pw'], ssl = True, database='fxtrading_md')
db_fxtrading_md.query('show measurements')

In [ ]:
crosslist = ['AUDCAD', 'USDSGD', 'AUDCNH', 'AUDSGD', 'CHFJPY', 'EURHUF', 'MXNJPY', 'CHFDKK', 'EURGBP', 'EURDKK', 'AUDNZD', 'CHFCNH', 'EURCZK', 'EURCNH', 'GBPUSD', 'GBPSEK', 'CADJPY', 'EURCHF', 'GBPNZD', 'GBPNOK', 'EURCAD', 'GBPMXN', 'GBPJPY', 'AUDCHF', 'AUDJPY', 'EURAUD', 'GBPDKK', 'DKKSEK', 'GBPCNH', 'GBPCHF', 'CADCNH', 'DKKNOK', 'GBPCAD', 'USDSEK', 'GBPAUD', 'USDRUB', 'USDPLN', 'DKKJPY', 'EURUSD', 'USDNOK', 'USDMXN', 'EURSGD', 'USDJPY', 'CADCHF', 'CNHJPY', 'EURSEK', 'USDILS', 'USDHUF', 'EURRUB', 'USDDKK', 'EURPLN', 'USDCZK', 'USDCNH', 'EURNZD', 'USDCHF', 'USDCAD', 'AUDUSD', 'CHFSEK', 'EURNOK', 'SGDJPY', 'SGDCNH', 'EURMXN', 'SEKJPY', 'NZDUSD', 'CHFNOK', 'EURJPY', 'NZDJPY', 'NZDCHF', 'EURILS', 'NOKSEK', 'NOKJPY']

# All the combinations

comb = [comb for comb in combinations(crosslist, 2)]
len(comb)

In [ ]:
class fx_data(object):
    """
    Extract the fx data from database
    """
    def __init__(self, crosslist, start, end):
        self.start = str(pd.to_datetime(start)).replace(' ', 'T') + 'Z'
        self.end = str(pd.to_datetime(end)).replace(' ', 'T') + 'Z'
        df_fx = pd.DataFrame()
        for i in crosslist:
            qline = "SELECT MidClose FROM md_rtbars WHERE time > '%s' AND time < '%s' AND Cross = '%s'"%(self.start, self.end, i)
            df_fx = pd.concat([df_fx, list(db_fxtrading_md.query(qline).values())[0].rename(columns={'MidClose': i})], axis = 1)
        self.df_fx = df_fx

In [ ]:
test = fx_data(crosslist, '2017-4-5', '2017-4-6')

In [ ]:
df = test.df_fx
pairs = '%s_%s'%(comb[0][0], comb[0][1])
pd.DataFrame({pairs: pd.rolling_corr(df[comb[0][0]], df[comb[0][1]], 18)}).head()

In [ ]:
import ipyparallel
clients = ipyparallel.Client()
dview = clients[:]
crosslist = test.df_fx.columns.tolist()
comb = [comb for comb in combinations(crosslist, 2)]

def chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

comb_p = list(chunks(comb, 311))

class fx_corr_para(object):
    """
    Calculate correlation from fx data
    """
    def __init__(self, df_fx, comb, window = 18):
        
        df_corr = pd.DataFrame()
        for i in comb:
            pairs = '%s_%s'%(i[0], i[1])
            df = pd.DataFrame({pairs: pd.rolling_corr(df_fx[comb[0][0]], df_fx[comb[0][1]], window)})
            df_corr = pd.concat([df_corr, df], axis = 1)
        self.df_corr = df_corr.reset_index()